# LIMPIEZA TABLA PROVEEDORES Y CARGA A SILVER

In [0]:
from pyspark.sql.functions import col, regexp_replace, trim, when, current_timestamp, length

# 1. Cargar desde Bronze
df_bronze_prov = spark.read.table("workspace.bronze.proveedores")

# 2. PROCESAMIENTO SILVER (Adaptación lógica Power BI)
df_silver_prov = df_bronze_prov.select(
    col("tipo_idenitificaci").alias("tipo_identificacion"),
    col("ci_ruc").alias("ruc"),
    col("razon_social"),
    col("direccion"),
    # Paso 8: Limpiar teléfonos (Solo dígitos)
    regexp_replace(col("telefono_1"), r'[^0-9]', '').alias("telefono"),
    col("email"),
    col("nombre_comercial"),
    col("tipo_contribuyent").alias("tipo_contribuyente"),
    # Paso 6: Normalizar booleanos
    when(col("ag_ret") == "Si", "Sí").otherwise(col("ag_ret")).alias("agente_retencion"),
    # Paso 7: Corregir Forma de Pago
    when((col("forma_de_pago").isNull()) | (trim(col("forma_de_pago")) == ""), "No Especificado")
    .otherwise(col("forma_de_pago")).alias("forma_pago")
).filter(
    # Paso 5: Filtrar RUC válido y evitar filas de encabezado repetidas o basura
    (col("tipo_identificacion") == "RUC") & 
    (col("ruc").isNotNull()) & 
    (trim(col("ruc")) != "") &
    (~col("razon_social").contains(";;;"))
).filter(
    # Paso 8 (continuación): Validar longitud mínima de teléfono
    (length(col("telefono")) >= 7) | (col("telefono").isNull())
)

# Paso 9 y 10: Manejar valores nulos en textos y tipos finales
df_silver_prov = df_silver_prov.fillna({
    "email": "", 
    "nombre_comercial": "",
    "direccion": "No Especificada"
})

# Paso 11: Eliminar duplicados por RUC
df_silver_prov = df_silver_prov.dropDuplicates(["ruc"]) \
    .withColumn("fecha_carga_silver", current_timestamp())

# 3. Guardar en Silver
df_silver_prov.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("workspace.silver.proveedores")

print("✅ Tabla Proveedores procesada en Silver con éxito.")
#display(spark.read.table("workspace.silver.proveedores").limit(10))